### The goal of this scraper to scape the sales table table from carsalesdatabase and load tables to mysql workbench
 * base_url: http://carsalesbase.com/us-car-sales-data/

In [1]:
# Import dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
from selenium import webdriver

from selenium.webdriver.support.ui import Select
driver = webdriver.Chrome()
from selenium.webdriver.common.by import By
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from mysql_conn import password
import datetime
import os

### Create an empty dataframe with car to store car make, model, and table html link, and picture link

In [2]:
car_links = pd.DataFrame()

car_links['Make'] = ""
car_links['Model'] = ""
car_links['table_url'] = ""
car_links['img_url'] = ""

car_links

,Make,Model,table_url,img_url


### Visit page and find all makes, models available in US sales section store in car links

In [3]:
# activate splinter

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
base_url = 'http://carsalesbase.com/us-car-sales-data/'
browser.visit(base_url)

time.sleep(5)

# Search for car
car_brands_search = browser.find_by_id('brand-option')
car_brands_search.click()



# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")
makes = soup.select('[name=brand-option] option[value]')
make_values = [make.get('value') for make in makes]
brands_textValues = [model.text for model in models]


### Get US Car Makes

In [8]:
car_makes = brands_textValues[2:-9]
car_makes.sort()
car_makes_unique = []
for i in car_makes:
    if i not in car_makes_unique:
        car_makes_unique.append(i)

In [9]:
car_links['Make'] = car_makes_unique
car_links.head()

,Make,Model,table_url,img_url
0,Acura,NaN,NaN,NaN
1,Alfa Romeo,NaN,NaN,NaN
2,Alpina,NaN,NaN,NaN
3,Alpine,NaN,NaN,NaN
4,Arcfox,NaN,NaN,NaN


In [11]:
car_sales = pd.DataFrame()
car_sales['Makes'] = car_makes_unique
car_sales.head(10)

,Makes
0,Acura
1,Alfa Romeo
2,Alpina
3,Alpine
4,Arcfox
5,Aston Martin
6,Audi
7,BAIC
8,BMW
9,BYD


### Next get Models

* Add code to click thru each make
    * Then select model from id = model-option
* Add model to to tables

### If make and model is predefined, use csv

In [7]:
table_links = pd.read_csv('database/table_links.csv')

table_links.head()

,make,model,Link
0,Chevrolet,Colorado,http://carsalesbase.com/us-car-sales-data/chev...
1,Jeep,Wrangler,http://carsalesbase.com/us-car-sales-data/jeep...
2,Chevrolet,Silverado,http://carsalesbase.com/us-car-sales-data/chev...
3,Subaru,WRX,http://carsalesbase.com/us-car-sales-data/suba...
4,GMC,Canyon,http://carsalesbase.com/us-car-sales-data/gmc/...


### Loop thru tables in each Link and capture tables
* Need to store each table as browser loops thru

In [10]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
tables = []

for i in table_links['Link']:
    
    try:
        link = i
        print(f'Processing data for: {link}')
        browser.visit(link)
        time.sleep(5)
        tables = pd.read_html(link)
        
        
    
    except (KeyError):

        print("Data unavilable...skipping.")

Processing data for: http://carsalesbase.com/us-car-sales-data/chevrolet/chevrolet-colorado
Processing data for: http://carsalesbase.com/us-car-sales-data/jeep/jeep-wrangler
Processing data for: http://carsalesbase.com/us-car-sales-data/chevrolet/chevrolet-silverado
Processing data for: http://carsalesbase.com/us-car-sales-data/subaru/wrx-sti/
Processing data for: http://carsalesbase.com/us-car-sales-data/gmc/gmc-canyon
Processing data for: http://carsalesbase.com/us-car-sales-data/toyota/toyota-4runner
Processing data for: http://carsalesbase.com/us-car-sales-data/gmc/gmc-sierra
Processing data for: http://carsalesbase.com/us-car-sales-data/toyota/toyota-tacoma
Processing data for: http://carsalesbase.com/us-car-sales-data/honda/honda-ridgeline
Processing data for: http://carsalesbase.com/us-car-sales-data/toyota/toyota-tundra
Processing data for: http://carsalesbase.com/us-car-sales-data/bmw/bmw-7-series
Processing data for: http://carsalesbase.com/us-car-sales-data/fiat/fiat-500/
Pr

### Add code to capture each table. 
* Previous table is showed each time code iterates to next car
* Need to handle tables with different date values
    * e.g. some tables only have 2018 data, some have 2019 data

### Clean data then read to database

In [ ]:
# take 1 of 4 in list
yearly_table.columns = tables[0].iloc[0]

# fill null values
yearly_table_2 = yearly_table.fillna('null')

# Create Month Column
yearly_table_2.rename(columns={ yearly_table_2.columns[0]: "Month"}, inplace=True)

# Capture make_name by stripping off year
make_name = yearly_table_2.columns[1][:-4].strip()

# Remove duplicate row
yearly_final = yearly_table_2.iloc[2:]

# Create 'Make Column'
yearly_final.insert(loc = 0, column = 'Make', value = make_name, allow_duplicates=False)

# Create empty header list varible
header = []

# Create new column headers using years (2015,2016,2017,etc.)
new_columns = yearly_final.columns[2:].values
for column in new_columns:
    header.append(column[-4:])
    
# Create new dataframe with year columns and values
yearly = yearly_final.rename(columns={ yearly_final.columns[2]: header[0],
                                      yearly_final.columns[3]: header[1],
                                      yearly_final.columns[4]:header[2],
                                      yearly_final.columns[5]:header[3]
                                      
                                  })
# make a copy dataframe copy
year_sale = yearly
# replace decimal point values with comma to denote thousands (U.S. units of cars sold)
year_sale['2015'] = [x.replace('.', ',') for x in year_sale['2015']]
year_sale['2016'] = [x.replace('.', ',') for x in year_sale['2016']]
year_sale['2017'] = [x.replace('.', ',') for x in year_sale['2017']]
year_sale['2018'] = [x.replace('.', ',') for x in year_sale['2018']]

In [12]:
# CREATE DATABASE connection
#password = os.environ['MY_SQL_PW']
rds_connection_string = f"root:{password}@127.0.0.1/cardb_test"
engine = create_engine(f'mysql://{rds_connection_string}')

In [13]:
Base = declarative_base()
session = Session(bind=engine)

# Object relational mapping for our table
class CarSales(Base):
    __tablename__ = 'car_sales'
    id = Column(Integer, primary_key=True)
    model = Column(String(255))
    month = Column(String(50))
    Year_2015 = Column(Integer)
    Year_2016 = Column(Integer)
    Year_2017 = Column(Integer)
    Year_2018 = Column(Integer)
    Year_2019 = Column(Integer)

In [ ]:
c = 0
maxRows = 10000
start_time = datetime.datetime.now()
for index, row in yearly.iterrows():
    record = CarSales()
    record.model = row['Model']
    record.month = row['Month']
    record.Year_2015 = row['2015']
    record.Year_2016 = row['2016']
    record.Year_2017 = row['2017']
    record.Year_2018 = row['2018']
    record.Year_2019 = row['2019']
    session.add(record)
    session.commit()
    c = c + 1
    if c >= maxRows: break

end_time = datetime.datetime.now()
print(end_time - start_time)